In [0]:
import numpy as np 
import pandas as pd
import string
import re
import nltk

Loading Data

In [0]:
! wget https://github.com/amsurve/J050_sem6_nlp/blob/master/datasets/10k-german-news-articles.zip?raw=true

--2020-03-03 18:48:35--  https://github.com/amsurve/J050_sem6_nlp/blob/master/datasets/10k-german-news-articles.zip?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/amsurve/J050_sem6_nlp/raw/master/datasets/10k-german-news-articles.zip [following]
--2020-03-03 18:48:40--  https://github.com/amsurve/J050_sem6_nlp/raw/master/datasets/10k-german-news-articles.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/amsurve/J050_sem6_nlp/master/datasets/10k-german-news-articles.zip [following]
--2020-03-03 18:48:41--  https://media.githubusercontent.com/media/amsurve/J050_sem6_nlp/master/datasets/10k-german-news-articles.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.13

In [0]:
! mkdir newsarticles
! unzip 10k-german-news-articles.zip?raw=true -d newsarticles

Archive:  10k-german-news-articles.zip?raw=true
  inflating: newsarticles/Annotations.csv  
  inflating: newsarticles/Annotations_consolidated.csv  
  inflating: newsarticles/Articles.csv  
  inflating: newsarticles/Categories.csv  
  inflating: newsarticles/CrossValSplit.csv  
  inflating: newsarticles/Newspaper_Staff.csv  
  inflating: newsarticles/Posts.csv  


In [0]:
ann_df=pd.read_csv("newsarticles/Annotations.csv")
posts_df=pd.read_csv("newsarticles/Posts.csv")
cats_df=pd.read_csv("newsarticles/Categories.csv")
News_df=pd.read_csv("newsarticles/Newspaper_Staff.csv")
articles_df=pd.read_csv("newsarticles/Articles.csv")
cvsplit_df=pd.read_csv("newsarticles/CrossValSplit.csv")
consol_df=pd.read_csv("newsarticles/Annotations_consolidated.csv")

Merging Dataframes to get all the features in one.

In [0]:
df= pd.merge(posts_df, articles_df, on='ID_Article')
df.head()

,ID_Post,ID_Parent_Post,ID_Article,ID_User,CreatedAt,Status,Headline,Body_x,PositiveVotes,NegativeVotes,Path,publishingDate,Title,Body_y
0,1,NaN,1,9089,2003-04-23 14:52:41.870,deleted,NaN,NaN,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
1,2,NaN,1,29367,2003-11-04 16:21:57.850,online,"Newsletter ""DER STANDARD""",Ich bin begeistert von den STANDARD - Newslett...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
2,3,2.0,1,5095,2004-01-28 12:57:28.240,deleted,Auch begeistert...,... Aber momentan funktioniert das Abmelden od...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
3,4,3.0,1,1682,2004-02-03 20:32:39.123,deleted,Abmeldeprobleme,Es ist ganz einfach nervend!\r\nVor kurzem hab...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
4,5,NaN,1,3343,2004-03-02 11:37:44.100,online,NaN,und sie als mitarbeiter sind natuerlich objektiv,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."


In [0]:
df= pd.merge(df, ann_df, on='ID_Post')
df.columns

Index(['ID_Post', 'ID_Parent_Post', 'ID_Article', 'ID_User', 'CreatedAt',
       'Status', 'Headline', 'Body_x', 'PositiveVotes', 'NegativeVotes',
       'Path', 'publishingDate', 'Title', 'Body_y', 'ID_Annotator', 'Category',
       'Value'],
      dtype='object')

In [0]:
df=pd.merge(df, cvsplit_df, on='ID_Post')
df.columns

Index(['ID_Post', 'ID_Parent_Post', 'ID_Article', 'ID_User', 'CreatedAt',
       'Status', 'Headline', 'Body_x', 'PositiveVotes', 'NegativeVotes',
       'Path', 'publishingDate', 'Title', 'Body_y', 'ID_Annotator',
       'Category_x', 'Value', 'Category_y', 'Fold'],
      dtype='object')

In [0]:
df= pd.merge(df, consol_df, on='ID_Post')
df.head()

,ID_Post,ID_Parent_Post,ID_Article,ID_User,CreatedAt,Status,Headline,Body_x,PositiveVotes,NegativeVotes,Path,publishingDate,Title,Body_y,ID_Annotator,Category_x,Value_x,Category_y,Fold,Category,Value_y
0,79,NaN,1,12071,2015-06-01 08:58:32.363,online,NaN,"ich kann keinen hinweis finden, wo man sich hi...",0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",1,PossiblyFeedback,1,PossiblyFeedback,1,PossiblyFeedback,1
1,81,NaN,1,27465,2015-10-03 14:06:06.210,online,NaN,Sehr geehrte Community oder cmb!\r\n\r\nBekomm...,0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",1,PossiblyFeedback,1,PossiblyFeedback,1,PossiblyFeedback,1
2,132,NaN,2,28073,2015-10-09 11:33:07.227,online,Irgendwie...,kommen manche Posts nicht durch :(,0,0,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",1,PossiblyFeedback,1,PossiblyFeedback,1,PossiblyFeedback,1
3,134,NaN,2,20194,2015-11-04 13:29:34.930,online,NaN,Eine Frage: \r\n\r\nGibt es eine Möglichkeit s...,0,0,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",1,PossiblyFeedback,1,PossiblyFeedback,1,PossiblyFeedback,1
4,139,NaN,2,14020,2015-12-11 14:51:17.540,online,Registrierung,"Ich habe ein Problem: ich bin registriert, übe...",0,0,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",1,PossiblyFeedback,1,PossiblyFeedback,1,PossiblyFeedback,1


In [0]:
df.shape

(4089860, 21)

Preprocessing Pipeline:

In [0]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
stop_words= set(stopwords.words('german'))

In [0]:
tokenizer= RegexpTokenizer(pattern= r"\s|[\.,;']", gaps=True)

In [0]:
regex = re.compile(r'(<[A-Za-z][A-Za-z0-9]*>|<\/?[^>]*>)')

In [0]:
def preprocess(str1):
    str1= str1.lower()
    str1= re.sub(regex, '', str1)
    word_tokens= tokenizer.tokenize(str1)
    filtered= [w for w in word_tokens if not w in stop_words]
    final= ''
    for word in filtered:
        if word.isalpha():
            final= final + ' ' + word
    final= final.translate(str.maketrans('', '', string.punctuation))
    return final

In [0]:
df['Body_y'][0]

'<div class="section" id="content-main" itemprop="articleBody"><div class="copytext"><h2 itemprop="description">Abonnieren Sie einen unserer Newsletter und lassen Sie sich aktuelle Nachrichten per Mail zuschicken! derStandard.at bietet eine Vielzahl themenbezogener Newsletter an.</h2><p>Unter dem Motto "Nachrichten in Echtzeit" liefert derStandard.at die wichtigsten Meldungen des Tages per E-Mail in einer HTML- oder Textversion. Unter <strong><a href="/Newsletter">derStandard.at/Newsletter</a></strong> können Sie Ihr kostenloses Newsletter-Abonnement bequem an- und abmelden.</p>\n<p><strong>Alle Newsletter im Überblick</strong></p>\n<ul><ul><li>D-Mail <strong>Newsroom:</strong> Spitzenmeldungen aus allen Newsroom-Ressorts</li><li>D-Mail <strong>Meinung:</strong> Aktuelle Kommentare und Kolumnen</li><li>D-Mail <strong>Internationalpolitik:</strong> Top-News über das internationale Geschehen</li><li>D-Mail <strong>Innenpolitik:</strong> Top-News über das innenpolitische Geschehen</li><li

In [0]:
preprocess(df['Body_y'][0])

' abonnieren unserer newsletter lassen aktuelle nachrichten per mail derstandard at bietet vielzahl themenbezogener newsletter motto liefert derstandard at wichtigsten meldungen tages per textversion derstandard kostenloses bequem abmelden newsletter überblick spitzenmeldungen aktuelle kommentare internationale innenpolitische nachrichten finanz neueste netz spitzenmeldungen meldungen chronik gesellschaft umwelt medien werbung sowie aktuelle brandaktuelle meldungen besonderer wichtigkeit liefert wöchentliche newsletter informiert jeweils wochenmitte neuigkeiten rund angebot derstandard at bringt highlights wöchentliche überblick aktuellen wöchentliche überblick wöchentliche news essen mode berichte tipps rund ums karrieretipps arbeitsleben neueste welt wöchentliche news rund autos beiträge user infos aktuellen gewinnspielen link login registrierung verwalten'

In [0]:
dff= df[:10000]

In [0]:
dff['Clean Body']= dff['Body_y'].apply(preprocess)
dff['Clean Title']= dff['Title'].apply(preprocess)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
dff.head() 

,ID_Post,ID_Parent_Post,ID_Article,ID_User,CreatedAt,Status,Headline,Body_x,PositiveVotes,NegativeVotes,Path,publishingDate,Title,Body_y,ID_Annotator,Category_x,Value_x,Category_y,Fold,Category,Value_y,Clean Body,Clean Title
0,79,NaN,1,12071,2015-06-01 08:58:32.363,online,NaN,"ich kann keinen hinweis finden, wo man sich hi...",0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",1,PossiblyFeedback,1,PossiblyFeedback,1,PossiblyFeedback,1,abonnieren unserer newsletter lassen aktuelle...,newsletter derstandard at
1,81,NaN,1,27465,2015-10-03 14:06:06.210,online,NaN,Sehr geehrte Community oder cmb!\r\n\r\nBekomm...,0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",1,PossiblyFeedback,1,PossiblyFeedback,1,PossiblyFeedback,1,abonnieren unserer newsletter lassen aktuelle...,newsletter derstandard at
2,132,NaN,2,28073,2015-10-09 11:33:07.227,online,Irgendwie...,kommen manche Posts nicht durch :(,0,0,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",1,PossiblyFeedback,1,PossiblyFeedback,1,PossiblyFeedback,1,teil derstandard dezember function width heig...,teil derstandard
3,134,NaN,2,20194,2015-11-04 13:29:34.930,online,NaN,Eine Frage: \r\n\r\nGibt es eine Möglichkeit s...,0,0,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",1,PossiblyFeedback,1,PossiblyFeedback,1,PossiblyFeedback,1,teil derstandard dezember function width heig...,teil derstandard
4,139,NaN,2,14020,2015-12-11 14:51:17.540,online,Registrierung,"Ich habe ein Problem: ich bin registriert, übe...",0,0,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",1,PossiblyFeedback,1,PossiblyFeedback,1,PossiblyFeedback,1,teil derstandard dezember function width heig...,teil derstandard


In [0]:
dff.isnull().sum()

ID_Post              0
ID_Parent_Post    3629
ID_Article           0
ID_User              0
CreatedAt            0
Status               0
Headline          6799
Body_x             140
PositiveVotes        0
NegativeVotes        0
Path                 0
publishingDate       0
Title                0
Body_y               0
ID_Annotator         0
Category_x           0
Value_x              0
Category_y           0
Fold                 0
Category             0
Value_y              0
Clean Body           0
Clean Title          0
dtype: int64

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vectorizer = TfidfVectorizer()

In [0]:
body= vectorizer.fit_transform(dff['Clean Body'])

In [0]:
body_features=  vectorizer.get_feature_names()

In [0]:
body_df= pd.DataFrame(body.toarray(), columns= body_features)

In [0]:
body_df.head()

,ab,abbilden,abbringen,abdrehen,abend,abenteuern,abfluss,abfragen,abgeblasen,abgefangen,abgelassenem,abgelenkt,abgerechnet,abgesagt,abgeschickt,abgeschlossen,abgespielt,abgestiegen,abgestraft,abgeworben,abgeworbenen,ablauf,ablaufdatum,ablehnung,ablenken,ablenkung,ablinger,abläuft,ablösefrei,abmelden,abnehmen,abnehmkuren,abo,aboagye,abonnieren,absagen,absatz,abschaffen,abschaffung,abschied,...,österreich,österreicher,österreicherinnen,österreichische,österreichischen,österreichischer,österreichs,övp,überarbeiteter,überarbeitung,überblick,überblickwie,überbucht,übergriffig,überhaupt,überholt,überkommt,überlebende,überlebenden,überlebte,überlässt,übernahme,übernehmen,überprüfen,überraschende,überraschender,überrascht,überraschungen,überreden,überschwemmungen,übersicht,übersiedeln,übersät,übertragen,überwiegen,überwiegt,überzeugter,üblichen,übrigens,čssr
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.069051,0.0,0.0,0.000000,0.0,0.084072,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.243596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.069051,0.0,0.0,0.000000,0.0,0.084072,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.243596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.045098,0.0,0.043026,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.045098,0.0,0.043026,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.045098,0.0,0.043026,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
vectorize= TfidfVectorizer()

In [0]:
Title= vectorize.fit_transform(dff['Clean Title'])

In [0]:
title_features= vectorize.get_feature_names()

In [0]:
title= pd.DataFrame(Title.toarray(), columns= title_features)

In [0]:
title.head()

,abgeworben,achtelfinale,adolf,android,angst,app,apple,artikeln,asked,at,austria,austrian,auszeit,auto,batterien,bein,benutzen,beschlagnahmte,besten,besuch,brauerei,britin,chat,clinton,community,derby,derstandard,direkt,dracula,drohne,ehefrau,eichmann,eigentor,eingriff,empört,enrique,entscheiden,erfahrungen,ergebnisse,erhielt,...,sv,südkorea,tag,tausende,teil,telegram,trainer,trump,tschechen,umgehen,unbeliebter,ungenauigkeit,unmöglichkeit,usa,use,verletzt,verlängerung,vermisst,verpflichtet,vikings,virtuellen,volk,vukojevic,watch,watford,wegen,weiteres,wer,wide,wiener,wissenschaft,wochen,wollten,world,wär,zahlen,zahnlücke,zurück,zwei,österreich
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.51428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.507269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.51428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.507269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.603807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.79713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.603807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.79713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.603807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.79713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
df2= dff.drop(['Path', 'publishingDate', 'CreatedAt', 'Title', 'ID_Parent_Post', 'Headline', 'Body_x', 'Body_y', 
               'Category_x', 'Category_y', 'Clean Body', 'Clean Title'], axis=1)

In [0]:
body_df= pd.concat([body_df, df2], axis=1)

In [0]:
body_df.head()

,ab,abbilden,abbringen,abdrehen,abend,abenteuern,abfluss,abfragen,abgeblasen,abgefangen,abgelassenem,abgelenkt,abgerechnet,abgesagt,abgeschickt,abgeschlossen,abgespielt,abgestiegen,abgestraft,abgeworben,abgeworbenen,ablauf,ablaufdatum,ablehnung,ablenken,ablenkung,ablinger,abläuft,ablösefrei,abmelden,abnehmen,abnehmkuren,abo,aboagye,abonnieren,absagen,absatz,abschaffen,abschaffung,abschied,...,überblickwie,überbucht,übergriffig,überhaupt,überholt,überkommt,überlebende,überlebenden,überlebte,überlässt,übernahme,übernehmen,überprüfen,überraschende,überraschender,überrascht,überraschungen,überreden,überschwemmungen,übersicht,übersiedeln,übersät,übertragen,überwiegen,überwiegt,überzeugter,üblichen,übrigens,čssr,ID_Post,ID_Article,ID_User,Status,PositiveVotes,NegativeVotes,ID_Annotator,Value_x,Fold,Category,Value_y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.069051,0.0,0.0,0.000000,0.0,0.084072,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79,1,12071,online,0,0,1,1,1,PossiblyFeedback,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.069051,0.0,0.0,0.000000,0.0,0.084072,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81,1,27465,online,0,1,1,1,1,PossiblyFeedback,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.045098,0.0,0.043026,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,132,2,28073,online,0,0,1,1,1,PossiblyFeedback,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.045098,0.0,0.043026,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,134,2,20194,online,0,0,1,1,1,PossiblyFeedback,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.045098,0.0,0.043026,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,139,2,14020,online,0,0,1,1,1,PossiblyFeedback,1


In [0]:
title_df= pd.concat([title, df2], axis=1)

In [0]:
title_df.head()

,abgeworben,achtelfinale,adolf,android,angst,app,apple,artikeln,asked,at,austria,austrian,auszeit,auto,batterien,bein,benutzen,beschlagnahmte,besten,besuch,brauerei,britin,chat,clinton,community,derby,derstandard,direkt,dracula,drohne,ehefrau,eichmann,eigentor,eingriff,empört,enrique,entscheiden,erfahrungen,ergebnisse,erhielt,...,ungenauigkeit,unmöglichkeit,usa,use,verletzt,verlängerung,vermisst,verpflichtet,vikings,virtuellen,volk,vukojevic,watch,watford,wegen,weiteres,wer,wide,wiener,wissenschaft,wochen,wollten,world,wär,zahlen,zahnlücke,zurück,zwei,österreich,ID_Post,ID_Article,ID_User,Status,PositiveVotes,NegativeVotes,ID_Annotator,Value_x,Fold,Category,Value_y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.51428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.507269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79,1,12071,online,0,0,1,1,1,PossiblyFeedback,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.51428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.507269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81,1,27465,online,0,1,1,1,1,PossiblyFeedback,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.603807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,132,2,28073,online,0,0,1,1,1,PossiblyFeedback,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.603807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,134,2,20194,online,0,0,1,1,1,PossiblyFeedback,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.603807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,139,2,14020,online,0,0,1,1,1,PossiblyFeedback,1


In [0]:
from sklearn.decomposition import LatentDirichletAllocation

In [0]:
no_topics = 20

In [0]:
lda_body = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,
                                     random_state= 42).fit(body)

In [0]:
def display_topics(model, feature_names, no_top_words, list1):
    for topic_idx, topic in enumerate(model.components_):
        topic= " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])
        list1.append(topic)

In [0]:
no_top_words = 4
topics_body= []
display_topics(lda_body, body_features, no_top_words, topics_body)

In [0]:
topics_body

['antworten forum erscheinen lassen',
 'linkshänder erledigen arbeiten rechts',
 'vukojevic zagreb kahat mittelfeldspieler',
 'watford prödl vertrag englischen',
 'kommt mal all immer',
 'konventionalstrafe arbeitgeber wettbewerbswidrig verstoß',
 'postings community foren männer',
 'internet ging seite online',
 'kollegin spaß gar schlafen',
 'werbung überblick adblockers adblocker',
 'fpö spö politik övp',
 'konventionalstrafe arbeitgeber arbeitnehmer verstoß',
 'fpö häupl wahl steiermark',
 'schicken jahr erinnerungen instagram',
 'ea kritik frauenfußball spielerinnen',
 'urlaub strand immerhin seiten',
 'trotz wer junge alter',
 'gericht britin verwarnt birmingham',
 'whatsapp nachrichten per nachrichtenüberblick',
 'regierung frau ministerium zitierte']

In [0]:
lda_title = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,
                                random_state= 42).fit(Title)

In [0]:
no_top_words = 4
topics_title= []
display_topics(lda_title, title_features, no_top_words, topics_title)

In [0]:
topics_title

['abgeworben zahlen wer kind',
 'zahlen abgeworben wer kind',
 'wollten kind scheu teil',
 'derstandard at facebook messenger',
 'shitstorm spieler sexistischer eigentor',
 'fpö wiener hände spö',
 'frequently asked questions community',
 'schlimmsten hotelerlebnisse tag vukojevic',
 'leben gut kind wollten',
 'wide world erstes mal',
 'fair news use landtagswahlen',
 'dracula millionenflüsse juni quartiere',
 'poroschenkoversteher schweigen batterien benutzen',
 'abgeworben zahlen wer community',
 'wär start lernen umgehen',
 'linkshänder erfahrungen app android',
 'jahr ergebnisse gewordene entscheiden',
 'koalitionäre handlanger eingriff grenzen',
 'siegt salzburg verlängerung zwei',
 'jahr schiffsunglück immer penissen']

Visualization:

In [0]:
import gensim
from gensim import corpora

In [0]:
text_list = [i.split() for i in dff['Clean Body']]
len(text_list)

10000

In [0]:
dictionary = corpora.Dictionary(text_list)
dictionary.save('dictionary.dict')
print(dictionary)

Dictionary(5044 unique tokens: ['abmelden', 'abonnieren', 'aktuelle', 'aktuellen', 'angebot']...)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_list]
corpora.MmCorpus.serialize('corpus.mm', doc_term_matrix)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
Lda = gensim.models.ldamodel.LdaModel

In [0]:
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50)

In [0]:
ldamodel.save('topic.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
from gensim.models import LdaModel
loading = LdaModel.load('topic.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
diction = gensim.corpora.Dictionary.load('dictionary.dict')
corpus = gensim.corpora.MmCorpus('corpus.mm')
lda = gensim.models.LdaModel.load('topic.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 49.0MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=ac1244293534fec07069e5d3aacf108dfd414d4e242595af82544c4d4b07718a
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=b4d1bba8e6cbf69d140bbfee4af1564bb25ffd5836d8e442384d4be9cade12ee
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
import pyLDAvis.gensim
import gensim
pyLDAvis.enable_notebook()

In [0]:
data = pyLDAvis.gensim.prepare(lda, corpus, diction)
data

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.063905  0.159756       1        1  41.586792
6     -0.129460 -0.237785       2        1  17.329731
3     -0.317148  0.153074       3        1  14.395793
5     -0.360066 -0.002867       4        1  10.993268
0      0.148278  0.122934       5        1   5.912574
7      0.145066  0.029998       6        1   3.894862
4      0.137104  0.186828       7        1   3.366087
1      0.081078  0.013607       8        1   1.843049
8      0.155158 -0.131512       9        1   0.347807
9      0.076084 -0.294033      10        1   0.330036, topic_info=     Category          Freq                Term         Total  loglift  logprob
1863  Default  34912.000000                 fpö  34912.000000  30.0000  30.0000
1884  Default  21806.000000                 spö  21806.000000  29.0000  29.0000
1238  Default  28383.000000  konventionalstrafe  28383.000000  28.0000  28.0000
1905  Default  13095.000000                 övp  13095.000000  27.0000  27.0000
1168  Default  24019.000000         arbeitgeber  24019.000000  26.0000  26.0000
...       ...           ...                 ...           ...      ...      ...
445   Topic10     23.026043          geschichte   2718.660645   0.9425  -5.6846
1477  Topic10     20.770636               sagte   2220.669922   1.0417  -5.7877
1838  Topic10     18.018711               titel    308.116119   2.8747  -5.9299
1463  Topic10     18.867052              montag   4387.933105   0.2645  -5.8838
151   Topic10     18.191841               sehen   4116.708984   0.2919  -5.9203

[604 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
403       7  0.972898             ab
403       9  0.025270             ab
3152      2  0.999747  abgeschlossen
3694      3  0.915861      abtäusche
3694      4  0.083924      abtäusche
...     ...       ...            ...
623       9  0.011143     österreich
3077      4  0.999756   österreicher
1905      2  0.166319            övp
1905      4  0.833353            övp
1905     10  0.000305            övp

[809 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 7, 4, 6, 1, 8, 5, 2, 9, 10])

Cosine Similarity:

In [0]:
body_topicvalues= lda_body.transform(body)
title_topicvalues= lda_title.transform(title)

In [0]:
final= dff[['Clean Body', 'Clean Title', 'Category']].copy()

In [0]:
final.head()

,Clean Body,Clean Title,Category
0,abonnieren unserer newsletter lassen aktuelle...,newsletter derstandard at,PossiblyFeedback
1,abonnieren unserer newsletter lassen aktuelle...,newsletter derstandard at,PossiblyFeedback
2,teil derstandard dezember function width heig...,teil derstandard,PossiblyFeedback
3,teil derstandard dezember function width heig...,teil derstandard,PossiblyFeedback
4,teil derstandard dezember function width heig...,teil derstandard,PossiblyFeedback


In [0]:
final['Topic Body'] = body_topicvalues.argmax(axis=1)
final['Topic Title']= title_topicvalues.argmax(axis=1)

In [0]:
def bodymap(value):
  return topics_body[value]

In [0]:
def titlemap(value):  
  return topics_title[value]

In [0]:
final['Topic Body']= final['Topic Body'].apply(bodymap)

In [0]:
final['Topic Title']= final['Topic Title'].apply(titlemap)

In [0]:
final.head()

,Clean Body,Clean Title,Category,Topic Body,Topic Title
0,abonnieren unserer newsletter lassen aktuelle...,newsletter derstandard at,PossiblyFeedback,werbung überblick adblockers adblocker,derstandard at facebook messenger
1,abonnieren unserer newsletter lassen aktuelle...,newsletter derstandard at,PossiblyFeedback,werbung überblick adblockers adblocker,derstandard at facebook messenger
2,teil derstandard dezember function width heig...,teil derstandard,PossiblyFeedback,whatsapp nachrichten per nachrichtenüberblick,wär start lernen umgehen
3,teil derstandard dezember function width heig...,teil derstandard,PossiblyFeedback,whatsapp nachrichten per nachrichtenüberblick,wär start lernen umgehen
4,teil derstandard dezember function width heig...,teil derstandard,PossiblyFeedback,whatsapp nachrichten per nachrichtenüberblick,wär start lernen umgehen


In [0]:
from scipy.spatial import distance

In [0]:
tfidf_vectorizer12 = TfidfVectorizer(analyzer='word')
tfidf_vectorizer12.fit(pd.concat([final['Clean Title'], final['Topic Title']]))
vec121 = tfidf_vectorizer12.transform(final['Clean Title'])
vec122 = tfidf_vectorizer12.transform(final['Topic Title'])

In [0]:
tfidf_vectorizer23 = TfidfVectorizer(analyzer='word')
tfidf_vectorizer23.fit(pd.concat([final['Topic Title'], final['Topic Body']]))
vec232 = tfidf_vectorizer23.transform(final['Topic Title'])
vec233 = tfidf_vectorizer23.transform(final['Topic Body'])

In [0]:
tfidf_vectorizer13 = TfidfVectorizer(analyzer='word')
tfidf_vectorizer13.fit(pd.concat([final['Clean Title'], final['Topic Body']]))
vec131 = tfidf_vectorizer13.transform(final['Clean Title'])
vec133 = tfidf_vectorizer13.transform(final['Topic Body'])

In [0]:
similarity12 = []
similarity23 = []
similarity13 = []

In [0]:
for i in range(len(topics_body)):
  similarity12.append(distance.cosine(vec121[i].toarray(), vec122[i].toarray()))
  similarity23.append(distance.cosine(vec232[i].toarray(), vec233[i].toarray()))
  similarity13.append(distance.cosine(vec131[i].toarray(), vec133[i].toarray()))

In [0]:
avg_score= list()
for i in range(len(similarity12)):
  avg= (similarity12[i] +similarity13[i] +similarity23[i])/3
  avg_score.append(avg)

In [0]:
Cosine_Similarities = pd.DataFrame(list(zip(similarity12, similarity13, similarity23, avg_score)), 
                                   columns=['Similarity12', 'Similarity13', 'Similarity23', 'Average Score'])

In [0]:
Cosine_Similarities

,Similarity12,Similarity13,Similarity23,Average Score
0,0.508175,1.000000,1.00000,0.836058
1,0.508175,1.000000,1.00000,0.836058
2,1.000000,1.000000,1.00000,1.000000
3,1.000000,1.000000,1.00000,1.000000
4,1.000000,1.000000,1.00000,1.000000
5,1.000000,1.000000,1.00000,1.000000
6,1.000000,1.000000,1.00000,1.000000
7,1.000000,1.000000,1.00000,1.000000
8,1.000000,1.000000,1.00000,1.000000
9,0.345820,0.708749,0.80121,0.618593


Similarity Scores:

In [0]:
def unique_words(str1):
  str1= str1.lower()
  word_tokens= tokenizer.tokenize(str1)
  filtered= [w for w in word_tokens if not w in stop_words]
  return set(filtered)

In [0]:
final['Unique Title']= final['Clean Title'].apply(unique_words)
final['Unique Topic Body']= final['Topic Body'].apply(unique_words)
final['Unique Topic Title']= final['Topic Title'].apply(unique_words)

In [0]:
bow_title= set()
bow_tbody= set()
bow_ttitle= set()

In [0]:
for i in range (final.shape[0]):
  bow_title= bow_title | (final['Unique Title'][i])
  bow_tbody= bow_tbody | (final['Unique Topic Body'][i])
  bow_ttitle= bow_ttitle | (final['Unique Topic Title'][i])

In [0]:
from itertools import combinations

In [0]:
combi_title = list(combinations(bow_title, 2))
combi_tbody = list(combinations(bow_tbody, 2))
combi_ttitle = list(combinations(bow_ttitle, 2))

In [0]:
from gensim.models import Word2Vec
from nltk.util import ngrams

In [0]:
corpus_title = list(ngrams(bow_title, 5))
corpus_tbody = list(ngrams(bow_tbody, 5))
corpus_ttitle = list(ngrams(bow_ttitle, 5))

In [0]:
model1 = Word2Vec(corpus_title, size=100, window=5, min_count=1, workers=4)
model2 = Word2Vec(corpus_tbody, size=100, window=5, min_count=1, workers=4)
model3 = Word2Vec(corpus_ttitle, size=100, window=5, min_count=1, workers=4)

In [0]:
similarity_title = []
similarity_tbody = []
similarity_ttitle = []

In [0]:
for i in range(0, 30):
  similarity_title.append(model1.similarity(combi_title[i][0], combi_title[i][1]))
  similarity_tbody.append(model2.similarity(combi_tbody[i][0], combi_tbody[i][1]))
  similarity_ttitle.append(model3.similarity(combi_ttitle[i][0], combi_ttitle[i][1]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/i

In [0]:
Combination_Similarities = pd.DataFrame(list(zip(combi_title, similarity_title, combi_tbody, similarity_tbody, combi_ttitle, similarity_ttitle)), 
                                        columns=['Combinations Title', 'Similarity Title', 'Combinations Topic Body', 'Similarity Topic Body', 
                                                 'Combinations Topic Title', 'Similarity Topic Title'])

In [0]:
Combination_Similarities

,Combinations Title,Similarity Title,Combinations Topic Body,Similarity Topic Body,Combinations Topic Title,Similarity Topic Title
0,"(russische, fleisch)",-0.167387,"(zitierte, jahr)",0.018014,"(tag, jahr)",0.098522
1,"(russische, direkt)",0.096349,"(zitierte, strand)",-0.119026,"(tag, wiener)",0.116424
2,"(russische, neuer)",-0.033086,"(zitierte, konventionalstrafe)",-0.110251,"(tag, facebook)",0.007755
3,"(russische, kerry)",0.003478,"(zitierte, junge)",0.084726,"(tag, grenzen)",-0.004103
4,"(russische, iglesias)",0.020366,"(zitierte, spielerinnen)",-0.061944,"(tag, asked)",0.091063
5,"(russische, wiener)",0.248591,"(zitierte, kommt)",-0.108113,"(tag, koalitionäre)",0.101944
6,"(russische, adolf)",0.159944,"(zitierte, trotz)",-0.023048,"(tag, fpö)",-0.018129
7,"(russische, perfekte)",-0.037170,"(zitierte, watford)",0.040305,"(tag, erfahrungen)",0.148628
8,"(russische, scheiterte)",0.023010,"(zitierte, immer)",0.083487,"(tag, fair)",-0.056304
9,"(russische, kolvidsson)",-0.029923,"(zitierte, lassen)",0.000749,"(tag, quartiere)",0.030134
